In [ ]:
import os
import pickle
import utils
from tqdm import tqdm

In [ ]:
try:
    os.mkdir('karobar-daily/')
    os.mkdir('karobar-daily/news')
    os.mkdir('karobar-daily/titles')
except Exception as e:
    print(e)

In [ ]:
BASE_URL = 'https://www.karobardaily.com/news/'
categories = [
    'economy',
    'banking',
    'insurance',
    'share_bazaar',
    'corporate',
    'country',
    'politics',
    'society',
    'development',
    'idea',
    'interview',
    'blog',
]

In [ ]:
for category in categories:
    # Get the url for each category
    url = BASE_URL + category
    page = 1
    print(f'####{category.upper()}####')
    while True:
        # Getting the HTML of the main page which contains articles
        print('Page:', page)
        try:
            page_url = f'{url}?page={page}'
            soup = utils.get_page_soup(page_url)
            texts = []
            titles = []
            # Get a list of all divs that contain the article header and the url for each article
            divs = soup.find_all('div', {'class': 'mn-header'})
            count = 1
            # For each article, do the following
            for div in tqdm(divs):
                try:
                    # Get the text
                    title = div.a.text
                    # Get the url
                    article_url = div.a['href']
                    
                    # Get the HTML of the article page ,which contains the actual content
                    article_soup = utils.get_page_soup(article_url)
                    
                    # Get the content div
                    text = article_soup.find('div', {'class': 'row main-data-content'})
                    
                    # If no such div found, go to another article
                    if text is None:
                        break
                    # Extract the article text    
                    text = text.text
                    text = utils.clean(text)
                    titles.append(title)
                    texts.append(text)

                except Exception as e:
                    print(f'===ARTICLE ERROR: {page}-{count}===\n\t### {e} ###')
                finally:
                    count += 1
            # write data to the respective files
            title_file = open(f'karobar-daily/titles/{category}-{page}.txt', 'wb')
            news_file = open(f'karobar-daily/news/{category}-{page}.txt', 'wb')       
            pickle.dump(titles, title_file)
            pickle.dump(texts, news_file)
            title_file.close()
            news_file.close()
            
        except Exception as e:
            print(f'===PAGE ERROR: {page}===\n\t### {e} ###')
        finally:
            page += 1
    break